## Imports

In [5]:
import numpy as np
import pandas as pd
import sklearn as sklearn
from sklearn.ensemble import HistGradientBoostingClassifier
print('Now using sklearn version '+sklearn.__version__)
print('Packages are Ready!!!')

Now using sklearn version 1.0.2
Packages are Ready!!!


## Loading data

In [6]:
print('Loading data...')
"""we are also importing training and testing data from competition because they might become handy"""

test=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/test.csv')
train=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/train.csv')

"""reading data"""

x_test=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/x_test.csv')
x_train=pd.read_csv('/Users/juansmacbook/PycharmProjects/Santander_Transaction/Santander-Transaction-Competition/x_train.csv')

print('Data is ready!!!!')


Loading data...
Data is ready!!!!


In [7]:
x_train.shape

(200000, 401)

In [8]:
x_train[x_train.columns[1:]]

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190_unique?,var_191_unique?,var_192_unique?,var_193_unique?,var_194_unique?,var_195_unique?,var_196_unique?,var_197_unique?,var_198_unique?,var_199_unique?
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,0,0,0,0,0,0,0,0,0,0
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,0,0,0,0,0,0,0,0,0,0
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,0,0,0,0,0,0,0,0,0,0
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,0,0,0,0,0,0,0,0,0,0
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,-0.6209,5.6669,...,0,1,0,0,0,0,1,0,0,0
199996,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,-3.6797,6.0019,...,0,0,0,1,0,0,0,0,0,0
199997,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,0.1202,6.1629,...,0,0,0,0,0,0,0,0,0,1
199998,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,-2.1926,8.7542,...,0,0,0,0,0,0,0,0,0,1


## Resampling
We are going to resample the missrepresented class in the training dataset, in this way we are going to balance out the classes.

In [9]:
training_set=x_train[x_train.columns[1:]].to_numpy()
training_labels=train['target'].to_numpy()


In [10]:

"""since we are dealing with an imbalanced data set we are going to deal with it  by oversampling the underrepresented class"""

from imblearn.over_sampling import RandomOverSampler
rov=RandomOverSampler()
x_resampled, y_resampled = rov.fit_resample(X=training_set, y=training_labels)

print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')
print('training set shape before resampling: '+str(training_set.shape))
print('training set shape after: '+str(x_resampled.shape))


print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')
print('training labels shape before resampling: '+str(training_labels.shape))
print('training labels shape after :'+str(y_resampled.shape))
print('_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._')

_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._
training set shape before resampling: (200000, 400)
training set shape after: (359804, 400)
_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._
training labels shape before resampling: (200000,)
training labels shape after :(359804,)
_.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._.-._


In [11]:
pd.Series(name='Resampled_Labels', data=y_resampled).value_counts()

0    179902
1    179902
Name: Resampled_Labels, dtype: int64

## Cross Validation


In [12]:
"""initializes an instance of a gradient boosting classifier """
def get_hgbc():
    hgbc=sklearn.ensemble.HistGradientBoostingClassifier(
        #Hyper parameters
        max_iter=500, #Trees to be build
        early_stopping=True,
        #validation_fraction=0.02,
        learning_rate=0.1,
        warm_start=True,
        #Metrics and over-fitting
        loss='binary_crossentropy',
        l2_regularization=0.01,
        scoring='roc_auc',
        # Tree parameters
        min_samples_leaf=120,
        max_leaf_nodes=13,
        max_depth=None,
        #ETC
        verbose=0
    )
    return hgbc
model=get_hgbc()

In [ ]:
from sklearn.model_selection import cross_val_score
val_scores=cross_val_score(model, x_resampled, y_resampled, cv=5, scoring='roc_auc')


## Let us train the model

In [28]:
"""Creates and fits model with the resampled data"""

model.fit(x_resampled, y_resampled, sample_weight=None)
print('Training done!')


Binning 1.128 GB of training data: 8.553 s
Binning 0.023 GB of validation data: 0.146 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 13 leaves, max depth = 8, train score: 0.65401, val score: 0.65667, in 0.210s
[2/1000] 1 tree, 13 leaves, max depth = 10, train score: 0.66526, val score: 0.66718, in 0.207s
[3/1000] 1 tree, 13 leaves, max depth = 11, train score: 0.68008, val score: 0.68676, in 0.212s
[4/1000] 1 tree, 13 leaves, max depth = 8, train score: 0.68902, val score: 0.69284, in 0.190s
[5/1000] 1 tree, 13 leaves, max depth = 9, train score: 0.69719, val score: 0.70223, in 0.192s
[6/1000] 1 tree, 13 leaves, max depth = 11, train score: 0.70382, val score: 0.70817, in 0.182s
[7/1000] 1 tree, 13 leaves, max depth = 12, train score: 0.71174, val score: 0.71894, in 0.191s
[8/1000] 1 tree, 13 leaves, max depth = 8, train score: 0.72016, val score: 0.72810, in 0.225s
[9/1000] 1 tree, 13 leaves, max depth = 8, train score: 0.72681, val score: 0.73470, in 0.193s
[10/1000] 1 tree, 13

In [47]:
"""Let's make predictions with the model"""
super_test_set=x_test[x_test.columns[1:]].to_numpy()
predictions=model.predict(super_test_set)
#TEST
predictions_prob=model.predict_proba(super_test_set)
predictions_prob=pd.DataFrame(predictions_prob)
######
predictions_prob

,0,1
0,0.512701,0.487299
1,0.557746,0.442254
2,0.764742,0.235258
3,0.668112,0.331888
4,0.278900,0.721100
...,...,...
199995,0.933356,0.066644
199996,0.153886,0.846114
199997,0.487547,0.512453
199998,0.895747,0.104253


In [48]:



"""Let's get ready for submission. Creating a DataFrame containing ID codes and targets"""
submission_df=pd.DataFrame()
submission_df['ID_code']=x_test['ID_code']

#TEST
#submission_df['target']=predictions THIS IS THE REAL DEAL
submission_df['target']=predictions_prob[1].values #TEST
########################

"""we are also creating a dummy columns to help us sort our data by ID code"""
submission_df['dummy_index']=submission_df['ID_code'].apply(lambda x: int(x[5:]))

submission_df

"""Let's sort our data and save it as a CSV file! We are done!"""
juan_submission=submission_df.sort_values(by='dummy_index')
juan_submission.drop(['dummy_index'],axis=1).to_csv('juan_submission.csv', index=False)
final_submission=pd.read_csv(filepath_or_buffer='juan_submission.csv')

In [52]:
!ls

Boosting copy.ipynb       juan_submission.csv       x_test.csv
DataGeneration copy.ipynb test.csv                  x_train.csv
README.md                 train.csv


In [51]:
final_submission

,ID_code,target
0,test_0,0.503196
1,test_1,0.791701
2,test_2,0.769796
3,test_3,0.487299
4,test_4,0.378554
...,...,...
199995,test_199995,0.320715
199996,test_199996,0.065481
199997,test_199997,0.104253
199998,test_199998,0.472951


Boosting copy.ipynb       juan_submission.csv       x_test.csv
DataGeneration copy.ipynb test.csv                  x_train.csv
README.md                 train.csv
